In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import tqdm

import warnings
warnings.simplefilter('ignore')

## 事前準備

In [9]:
list_df = pd.DataFrame(columns=['アーティスト', '曲名','歌詞', 'ラベル'])

def no2artist(no):
    if no == '12795':
        return '米津玄師', 0
    elif no == '18093':
        return 'Official髭男dism', 1
    elif no == '28370':
        return 'YOASOBI', 2
    elif no == '23343':
        return 'King Gnu', 3
    elif no == '17598':
        return 'あいみょん', 4
    elif no == '27857':
        return 'Vaundy', 5
    elif no == '28712':
        return '瑛人', 6
    elif no == '11591':
        return 'LiSA', 7
    elif no == '14957':
        return 'DISH//', 8

## 歌詞を取得

In [10]:
#歌ネットのurlをbase_urlに入力します
base_url = 'https://www.uta-net.com'
#usr_agentに先ほど取得したUserAgent情報を入力します
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
header = {'User-Agent': user_agent}

targets = ['12795', '18093', '28370', '23343', '17598', '27857', '28712', '11591', '14957']

for target in tqdm.tqdm(targets):
    #urlに先ほど取得した歌詞一覧のURLを入力します
    url = f'https://www.uta-net.com/artist/{target}/'
    artist, label = no2artist(target)

    response = requests.get(url,headers=header)

    soup = BeautifulSoup(response.text, 'lxml')

    #引数として、class_='sp-w-100'を与えます
    links = soup.find_all('td', class_='sp-w-100')

    #歌詞情報を取得します
    for link in links:
        a = base_url + (link.a.get('href'))
        response = requests.get(a)
        soup = BeautifulSoup(response.text, 'lxml')
        song_name = soup.find('h2').text
            
        song_kashi = soup.find('div', id="kashi_area")
        song_kashi = song_kashi.text
            
        time.sleep(1)
            
        tmp_se = pd.DataFrame([artist, song_name, song_kashi, label], index=list_df.columns).T
        
        list_df = list_df.append(tmp_se)

100% 9/9 [18:30<00:00, 123.37s/it]


## 確認

In [11]:
print(list_df.shape)
print(list_df)

(651, 4)
    アーティスト             曲名                                                 歌詞  \
0     米津玄師           LADY  例えば僕ら二人　煌めく映画のように出会いなおせたらどうしたい何も謎めいてない　今日は昨日の続...   
0     米津玄師        アイネクライネ  あたしあなたに会えて本当に嬉しいのに当たり前のようにそれらすべてが悲しいんだ今痛いくらい幸せ...   
0     米津玄師       あたしはゆうれい  真っ赤な林檎に小さなランプ灯りが射し込んだソファの上であなたはいつものように眠ってあたしは鏡...   
0     米津玄師       雨の街路に夜光蟲  震えていたのは君の手か僕の手の方か街のライトが雨に照らされて弾ける夜に僕らはただ大人になりた...   
0     米津玄師         あめふり婦人  街がびしゃびしゃ濡れる夜までは確かに声で溢れていた今ではおよそ耐え難いくらいの静けさ達がここ...   
..     ...            ...                                                ...   
0   DISH//         Replay  Replay巻き戻せない同じ朝は二度とないよReplay共に歩こうそのキセキを掴んでいよう咲...   
0   DISH//         理由のない恋  試着室のカーテン　開けた瞬間に君が「全然似合わない」って　笑って顔を伏せた好きなものならば　...   
0   DISH//           ルーザー  まず掲げるは人が人たらしめる爆弾発言ささて始めるか実在主義はなんたるか難解なもんか信じるもの...   
0   DISH//          Loop.  Good Morning, Good Night　毎日　ループ Down な状態　打開したい...   
0   DISH//  rock'n'roller  賽は投げられた告げられる合図勝ちなんかじゃなく価値一択の世界で知らないと捨てられたそのゴミ屑...   

   ラベル  
0    0  
0    0  
0  

In [12]:
list_df.to_csv('data.csv', index=False)